In [ ]:
import json
from datetime import datetime
from dateutil import parser
import pandas as pd
import calplot
import matplotlib.pyplot as plt
from urllib.parse import urlparse

In [ ]:
from pyecharts.charts import Bar, Calendar, Pie
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from snapshot_selenium import snapshot
from pyecharts.render import make_snapshot

In [ ]:
with open("aw-buckets-export-2022.json", "r") as f:
    data = json.load(f)

In [91]:
app_usage = {}
dayrecord = {}
total_duration = 0.0

for event in data['buckets']['aw-watcher-window_pc']['events']:
    date = parser.parse(event["timestamp"]).strftime("%Y-%m-%d")
    duration = event['duration']
    appname = event['data']['app']
    
    if "2022" in date:
        total_duration += duration
        if appname in app_usage:
            app_usage[appname] = duration + app_usage[appname]
        else:
            app_usage[appname] = duration

        if appname != 'LockApp.exe':
            if date in dayrecord:
                dayrecord[date] += duration
            else:
                dayrecord[date] = duration

In [89]:
sum_ = []
for event in data['buckets']['aw-watcher-window_pc']['events']:
    date = parser.parse(event["timestamp"]).strftime("%Y-%m-%d")
    if "2022-01-01" == date:
        duration = event['duration']
        if event['data']['app'] == 'LockApp.exe':
            continue
        sum_.append(duration)

print(sum(sum_))

67203.33200000001


In [ ]:
hours = total_duration//3600
print("Total PC Usage in hours: ", hours)
print("Total PC Usage in days: ", hours//24)

In [ ]:
app_usage_hourly = {}

for k,v in app_usage.items():
    usage_time = int(v//3600)
    if usage_time > 10:
        app_usage_hourly[k[:-4]] = usage_time

print(app_usage_hourly)

In [ ]:
pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.CHALK))

t = [list(z) for z in zip(app_usage_hourly.keys(), app_usage_hourly.values())]
pie.add("", t)
pie.set_global_opts(legend_opts=opts.LegendOpts(is_show=False), title_opts=opts.TitleOpts(title="Application Usage (in hours)",  pos_left="right",))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
# pie.render()
make_snapshot(snapshot, pie.render(), "pc_app_usage.png")

In [ ]:
apps_sorted = dict(sorted(app_usage_hourly.items(), key=lambda item: item[1], reverse=True))
apps_df = pd.DataFrame(apps_sorted.items(), columns=["Applications", "Total Hours"])
P = apps_df.plot(kind="barh", x=0, y=1, figsize=(10,6), color="orange")
P.set_facecolor('gray')
plt.show()

# Heatmap for PC usage

In [92]:
dataToPlot = []

for k,v in dayrecord.items():
    date_ = datetime.strptime(k, "%Y-%m-%d")
    if date_.year == 2022:
        dataToPlot.append([date_.strftime('%Y-%m-%d'),int(v)//3600])


In [76]:
dayrecord

{'2022-12-30': 6785.667000000001,
 '2022-12-29': 26506.036000000022,
 '2022-12-28': 49863.52099999993,
 '2022-12-27': 49515.58099999991,
 '2022-12-26': 32752.610999999975,
 '2022-12-24': 32599.438000000016,
 '2022-12-23': 35459.50099999997,
 '2022-12-22': 47570.28200000005,
 '2022-12-21': 46829.02200000017,
 '2022-12-20': 42778.801,
 '2022-12-19': 22980.47200000004,
 '2022-12-18': 37591.60600000007,
 '2022-12-17': 46514.557999999946,
 '2022-12-16': 48648.922000000006,
 '2022-12-15': 42098.96500000001,
 '2022-12-14': 37729.37499999995,
 '2022-12-13': 48005.75799999996,
 '2022-12-12': 48408.84799999999,
 '2022-12-11': 48203.139999999934,
 '2022-12-10': 49610.66,
 '2022-12-09': 50714.88799999987,
 '2022-12-08': 50583.65099999989,
 '2022-12-07': 41273.01899999996,
 '2022-12-06': 54424.57100000012,
 '2022-12-05': 49085.10499999997,
 '2022-12-04': 46666.123999999996,
 '2022-12-03': 51041.35500000009,
 '2022-12-02': 46571.057999999975,
 '2022-12-01': 45747.80300000003,
 '2022-11-30': 39727.10

In [94]:
cal = Calendar()
cal.add("", dataToPlot, calendar_opts=opts.CalendarOpts(range_="2022"))
cal.set_global_opts(
        title_opts=opts.TitleOpts(title="PC Usage", subtitle="Hours per day"),
        visualmap_opts=opts.VisualMapOpts(
            max_=40,
            min_=0,
            orient="horizontal",
            is_piecewise=True,
            pos_top="230px",
            pos_left="100px",
        ),
    )
# cal.render()

make_snapshot(snapshot, cal.render(), 'PC_usage_heatmap.png')

# Diving into Firefox

In [ ]:
website_record = {}
total_duration = 0


for event in data['buckets']['aw-watcher-web-firefox']['events']:
    date = parser.parse(event["timestamp"]).strftime("%Y-%m-%d")
    duration = event['duration']
    # website = event['data']['title']
    website = urlparse(event['data']['url']).netloc

    if "2022" in date:
        total_duration += duration
        if website in website_record:
            website_record[website] = duration + website_record[website]
        else:
            website_record[website] = duration


In [ ]:
website_hr_usage = {}

for k,v in website_record.items():
    usage_time = int(v//3600)
    if usage_time > 10:
        website_hr_usage[k] = usage_time

sorted_website_hr_usage = dict(sorted(website_hr_usage.items(), key=lambda item: item[1], reverse=True))


In [ ]:
pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.CHALK))
tj = [list(z) for z in zip(sorted_website_hr_usage.keys(), sorted_website_hr_usage.values())]
pie.add("", tj)
pie.set_global_opts(legend_opts=opts.LegendOpts(is_show=False), title_opts=opts.TitleOpts(title="Most Websites opened in firefox (in hours)",  pos_left="right",))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
# pie.render()
make_snapshot(snapshot, pie.render(), "firefox_websites.png")